In [8]:
import numpy as np
import matplotlib.pyplot as plt
import future.utils

class OpinionEvolution(object):
    def __init__(self, trends, p_o, p_p, kind='probability'):
        
        """
        :param p_o: The model p_o parameter
        :param p_p: The model p_p parameter
        :param trends: The computed simulation trends (status.json in the run folder)
        """
        
        self.kind = kind
        
        self.node2col = {}
        self.nodes2opinions = {}
            
        self.last_it = trends[-1]['iteration'] + 1
        self.last_seen = {}
        
        def transform(w: list, p_o: int, p_p: int):
            w = np.array(w)
            p = w * p_o + (1 - w) * p_p
            p = p.tolist()
            return p

        for it in trends:
            weights = np.array([el for el in it['status'].values()])
            if kind == 'probability':
                sts = transform(weights, p_o, p_p)  # update conditional probabilities of event will occur
            else:
                sts = weights
            its = it['iteration']
            for n, v in enumerate(sts):
                if n in self.nodes2opinions:
                    last_id = self.last_seen[n]
                    last_value = self.nodes2opinions[n][last_id]

                    for i in range(last_id, its):
                        self.nodes2opinions[n][i] = last_value

                    self.nodes2opinions[n][its] = v
                    self.last_seen[n] = its
                else:
                    self.nodes2opinions[n] = [0]*self.last_it
                    self.nodes2opinions[n][its] = v
                    self.last_seen[n] = 0
                    if v < 0.33:
                        self.node2col[n] = '#357db0'
                    elif 0.33 <= v <= 0.66:
                        self.node2col[n] = '#18A558'
                    else:
                        self.node2col[n] = '#ce2626'
    
    def plot(self, filename=None, ax = None):   
        if ax is None:
            fig, ax = plt.subplots(figsize=(10, 6))    
        mx = 0
        for k, l in future.utils.iteritems(self.nodes2opinions):
            if mx < self.last_seen[k]:
                mx = self.last_seen[k]
            x = list(range(0, self.last_seen[k]))
            y = l[0:self.last_seen[k]]
            ax.plot(x, y, lw=1.5, alpha=0.5, color=self.node2col[k])
        plt.xlabel("Iterations")
        plt.ylabel(f"{self.kind.capitalize()}")
        if filename is not None:
            plt.savefig(filename, dpi=300, facecolor='white', bbox_inches='tight')
        else:
            plt.show()
            
        plt.close()

In [9]:
n_lobbyists = 2

In [10]:
import json
with open(f'../results/{n_lobbyists}_lobbyistsconfig.json', 'r') as f:
    params = json.load(f)

In [11]:
print(params)

{'p_o': 0.01, 'p_p': 0.99, 'lambda_values': [0.0, 0.5, 1.0], 'phi_values': [0.0, 0.5, 1.0], 'T': 10000, 'n_lobbyists': 2, 'lobbyists': {'0': {'m': 0, 'strategy': ''}, '1': {'m': 1, 'strategy': ''}}, 'nruns': 3}


In [12]:
p_o = params['p_o']
p_p = params['p_p']
lambda_values = params['lambda_values']
phi_values = params['phi_values']
n_lobbyists = params['n_lobbyists']
lobbyists_data = params['lobbyists']
nruns = params['nruns']

In [13]:
type(lambda_values)

list

In [14]:
type(phi_values)

list

In [15]:
import json
import os 

values = ['weights', 'probabilities']

for value in values:
    for _, (lambda_v, phi_v) in enumerate([(l, p) for l in params['lambda_values'] for p in params['phi_values']]):    
        path = os.path.join(f'../results/{n_lobbyists}_lobbyists/', f'{lambda_v}_{phi_v}/')        
        for run in range(params['nruns']):
            runpath = os.path.join(path, str(run))
            with open(runpath+'/status.json', 'r') as f:
                trends = json.load(f)
            
            oe = OpinionEvolution(trends, p_o, p_p, kind=value)
            oe.plot(runpath+f'/{value}_evolution.png')